# Spotipy Demo

See link https://spotipy.readthedocs.io/en/2.7.1/

We can access generic spotify data such as album, artist, and playlist information. We can also access user information only if the user gives us authentication.

Note the an instance of a SpotifyUser needs to be associated with a specific Spotify developer app, as we create a SpotifyUser object by requesting authorization from a specific user for a specific app. When requesting this information, we need to have a set spotipy redirect uri. To request authorization for a developer app, you need to go into the settings for the developer app and change the redirect uri to the same thing as what you want to use for initiating SpotifyUser objects (such as http://localhost:8888).

### Possible scopes
#### Things we (probably) want
user-library-read allows access to user's music library, including the user's saved albums and tracks.

user-follow-read allows access to which artists a user follows. 

user-top-read allows us to get the user's top artists and tracks.

user-read-recently-played allows us to see recently played tracks.

#### Things we (probably) do not want
user-read-private gives permission to access personal subscription details, I don't think we want to ask for this (or that we should).

playlist-read-private allows us to see the playlists a user follows and check if a user follows a specific playlist. We probably will focus on songs and artists instead so won't need this permission.

In [1]:
# need to do "pip install spotipy" to have access to this packagea
import spotipy
import sys
import spotipy.util as util

In [2]:
spotify = spotipy.Spotify()

In [3]:
class SpotifyUser:
    """
    This class defines a Spotify user. A user is identified
    by their username and will need to give authorization under
    certain scopes for the other methods to be called. Once 
    authorization is given, multiple methods will obtain user 
    related information from Spotify.
    """
    
    #include scope in this function or no?
    def __init__(self, spotipy_client_id, spotipy_client_secret, spotipy_redirect_uri, username):
        self.username = username
        scope = 'user-library-read user-follow-read user-top-read user-read-recently-played'
        self.token = util.prompt_for_user_token(username, scope, spotipy_client_id, spotipy_client_secret, spotipy_redirect_uri)
        
        #verify that user authorization was successful
        if self.token:
            self.sp = spotipy.Spotify(auth=self.token)
        else:
            print("ERROR: User authorization unsuccessful.")
            #do anything else? raise an exception?
        
    def info(self):
        if self.token:
            return self.sp.me()
#             for item in results['items']:
#                 track = item['track']
#                 print(track['name'] + ' - ' + track['artists'][0]['name'])
        else:
            print("ERROR: Cannot get user info when user", self.username, "has not given authorization.")
            
    # Returns some of the most recently played tracks.
    # number_of_tracks controls how many tracks are returned.
    # DOES NOT CURRENTLY WORK - I am not sure why
    def get_most_recent_tracks(self, number_of_tracks=20):
        if self.token:
            return self.sp.current_user_recently_played(number_of_tracks)
#             for item in results['items']: #example of extracting name and artist of tracks
#                 track = item['track']
#                 print(track['name'] + ' - ' + track['artists'][0]['name'])
        else:
            print("ERROR: Cannot get most recent tracks when user", self.username, "has not given authorization.")

    # Returns a limited number of the user's saved tracks.
    # number_of_tracks controls how many tracks are returned at a time.
    # You can use the offset parameter to get songs starting after a certain song number.
    # This allows us to retrieve all songs by querying for a limited number of tracks at a
    # time and looping.
    def get_saved_tracks(self, number_of_tracks=20, offset=0):
        if self.token:
            return self.sp.current_user_saved_tracks(number_of_tracks, offset)
        else:
            print("ERROR: Cannot get saved tracks when user", self.username, "has not given authorization.")
            
    # Returns a limited number of the user's saved albums.
    # number_of_albums controls how many albums are returned at a time.
    # You can use the offset parameter to get albums starting after a certain song number.
    # This allows us to retrieve all albums by querying for a limited number of tracks at a
    # time and looping.
    def get_saved_albums(self, number_of_albums=20, offset=0):
        if self.token:
            return self.sp.current_user_saved_albums(number_of_albums, offset)
        else:
            print("ERROR: Cannot get saved albums when user", self.username, "has not given authorization.")
    
    # Returns a limited number of the artists followed by the current user.
    # number_of_artists controls how many artists are retrieved at a time.
    # You can use the after parameter to get artists starting after a certain artists id
    # To get all followed artists, you need to get x artists at a time, then get the next
    # x artists starting after the last artist in the previous query.
    def get_followed_artists(self, number_of_artists=20, after=None):
        if self.token:
            return self.sp.current_user_followed_artists(number_of_artists, after)
        else:
            print("ERROR: Cannot get followed artists when user", self.username, "has not given authorization.")
    
    # Returns the most popular tracks for the current user.
    # number_of_tracks controls how many tracks are returned.
    # time_range = {'short_term', 'medium_term', 'long_term'} controls whether we get the most 
    #  recent popular songs, somewhat recent popular songs, or long-term popular songs.
    # offset controls the starting point, allowing us to say get the 20-40 most popular tracks.
    #  I assume we will not need to use offset for our purposes so I set the default value to 0.
    def get_top_tracks(self, number_of_tracks=20, time_range='short_term', offset=0):
        if self.token:
            return self.sp.current_user_top_tracks(number_of_tracks, offset, time_range)
        else:
            print("ERROR: Cannot get top tracks when user", self.username, "has not given authorization.")        
    
    # Returns the most popular artists for the current user.
    # number_of_artists controls how many tracks are returned.
    # time_range = {'short_term', 'medium_term', 'long_term'} controls whether we get the most 
    #  recent popular artists, somewhat recent popular artists, or long-term popular artists.
    # offset controls the starting point, allowing us to say get the 20-40 most popular tracks.
    #  I assume we will not need to use offset for our purposes so I set the default value to 0.
    def get_top_artists(self, number_of_tracks=20, time_range='short_term', offset=0):
        if self.token:
            return self.sp.current_user_top_artists(number_of_tracks, offset, time_range)
        else:
            print("ERROR: Cannot get top artists when user", self.username, "has not given authorization.")  

In [4]:
"""
Test of initiating a SpotifyUser object.
For Ben's developer app, use
SPOTIPY_CLIENT_ID    ='46a21c22c2b84831a61e13aabc27dc30'
SPOTIPY_CLIENT_SECRET='3211a9953f4d4facbe8fad5d6b171ea2'
SPOTIPY_REDIRECT_URI  ='http://localhost:8888'
"""
new_username = 'CarolineWang'
new_user = SpotifyUser('46a21c22c2b84831a61e13aabc27dc30', '3211a9953f4d4facbe8fad5d6b171ea2', 'http://localhost:8888', new_username)

In [5]:
"""
Test prints from all methods I have written.
"""
print(new_user.info())
print(new_user.get_most_recent_tracks())
print(new_user.get_saved_tracks())
print(new_user.get_saved_albums())
print(new_user.get_followed_artists())
print(new_user.get_top_tracks())
print(new_user.get_top_artists())

{'display_name': 'Caroline Wang', 'external_urls': {'spotify': 'https://open.spotify.com/user/314xcqarki42gnkosjtfplluumya'}, 'followers': {'href': None, 'total': 2}, 'href': 'https://api.spotify.com/v1/users/314xcqarki42gnkosjtfplluumya', 'id': '314xcqarki42gnkosjtfplluumya', 'images': [{'height': None, 'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=437581406376376&height=300&width=300&ext=1584584487&hash=AeSVSbEbGc1xn7S8', 'width': None}], 'type': 'user', 'uri': 'spotify:user:314xcqarki42gnkosjtfplluumya'}
{'items': [{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'}, 'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q', 'id': '53XhwfbYqKCa1cC15pYq2q', 'name': 'Imagine Dragons', 'type': 'artist', 'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'}], 'available_markets': ['CA', 'MX', 'US'], 'external_urls': {'spotify': 'https://open.spotify.com/album/0gmsXcmr

{'href': 'https://api.spotify.com/v1/me/albums?offset=0&limit=20', 'items': [{'added_at': '2019-12-11T17:20:55Z', 'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5aIqB5nVVvmFsvSdExz408'}, 'href': 'https://api.spotify.com/v1/artists/5aIqB5nVVvmFsvSdExz408', 'id': '5aIqB5nVVvmFsvSdExz408', 'name': 'Johann Sebastian Bach', 'type': 'artist', 'uri': 'spotify:artist:5aIqB5nVVvmFsvSdExz408'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/5Dl3HXZjG6ZOWT5cV375lk'}, 'href': 'https://api.spotify.com/v1/artists/5Dl3HXZjG6ZOWT5cV375lk', 'id': '5Dl3HXZjG6ZOWT5cV375lk', 'name': 'Yo-Yo Ma', 'type': 'artist', 'uri': 'spotify:artist:5Dl3HXZjG6ZOWT5cV375lk'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW',

{'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'followers': {'href': None, 'total': 33023317}, 'genres': ['canadian pop', 'pop', 'post-teen pop'], 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', 'id': '1uNFoZAHBGtllmzznpCI3s', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/d745628f6659a715f0f69eed1508805849796e5e', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/5070bcb38e0fa03a21166a57cf93e4089effeec1', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/36235a9233a74be64a525f7998f0e94e78eb63e3', 'width': 160}], 'name': 'Justin Bieber', 'popularity': 98, 'type': 'artist', 'uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/6Sqhwn4JweupJ3sn9zHNsq'}, 'followers': {'href': None, 'total': 135}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/6Sqhwn4JweupJ3sn9zHNsq', 'id': '6Sqhwn4JweupJ3sn9zHNsq', 'images': [{

In [ ]:
""" information for using Selen's developer account app
username = "1214600613" #Selen's spotify username
scope    = "user-read-private"
client_id = "713a065a310b499c93af68eaefb213e7"; # Your client id
client_secret = "c2cdbfa0fcdc4bbb9461638c57a0a41e"; # Your secret
redirect_uri = "http://localhost:8888"; # Your redirect uri
util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)
"""

In [ ]:
""" information for using Ben's developer account app
username = "bnativi17" #Ben's spotify username
scope    = "user-read-private"
client_id = "713a065a310b499c93af68eaefb213e7"; # Your client id
client_secret = "c2cdbfa0fcdc4bbb9461638c57a0a41e"; # Your secret
redirect_uri = "http://localhost:8888"; # Your redirect uri
util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri='http://localhost:8888')
"""